In [ ]:
from IPython.display import Image
from rtmag.process.download.find_harp import find_ar

In [ ]:
# d = "2011-02-14 00:00:00"
d = "2024-01-06 11:48:00"
ar, img = find_ar(d, show_image=True)
print(ar)
Image(img)

In [ ]:
harpnum = 10603

In [ ]:
from rtmag.process.download.dl_map import get_sharp_map

In [ ]:
hmi_map, hmi_data = get_sharp_map(d, harpnum)

In [ ]:
hmi_map.peek()

In [ ]:
hmi_data.shape

In [ ]:
from rtmag.process.paper.hmi_to_input import get_input

In [ ]:
nx = 512
ny = 256

model_input, x, y, z, dx, dy, dz = get_input(hmi_data, nx, ny)
model_input.shape

In [ ]:
meta_path = "D:/models/uno/best_model.pt"

from rtmag.process.paper.load import MyModel
device = "cuda"

In [ ]:
mm = MyModel(meta_path, device=device)

In [ ]:
b = mm.get_pred_from_numpy(model_input)
b.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
bottom = b[:, :, 0, 2]

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(x, y, bottom.T, cmap='gray', vmin=-2000, vmax=2000)
ax.set_xlabel('x [Mm]')
ax.set_ylabel('y [Mm]')
ax.set_aspect('equal')
ax.set_title("PINO's output at z=0")
plt.tight_layout()
plt.show()

In [ ]:
mask = np.abs(bottom) > 500
fig, ax = plt.subplots()
ax.pcolormesh(x, y, (bottom*mask).T, cmap='gray', vmin=-2000, vmax=2000)
ax.set_xlabel('x [Mm]')
ax.set_ylabel('y [Mm]')
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

In [ ]:
stride = 20
seeds = np.stack([np.where(mask == True)[0], 
                  np.where(mask == True)[1], 
                  np.zeros_like(np.where(mask == True)[0])], axis=1)
seeds = seeds[::stride]
seeds.shape

In [ ]:
from streamtracer import StreamTracer, VectorGrid

In [ ]:
nsteps = 10000
step_size = 0.1
grid_spacing = [1, 1, 1]

grid = VectorGrid(b.astype(np.float64), grid_spacing)
tracer = StreamTracer(nsteps, step_size)
tracer_xs = []
tracer.trace(seeds, grid)
tracer_xs.append(tracer.xs)
tracer_xs = [item for sublist in tracer_xs for item in sublist]

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(x, y, bottom.T, cmap='gray', vmin=-2000, vmax=2000)
ax.set_xlabel('x [Mm]')
ax.set_ylabel('y [Mm]')
ax.set_aspect('equal')
for i, xl in enumerate(tracer_xs):
    assert seeds[i] in xl 
    ax.plot(xl[:,0]*dx+x[0], xl[:,1]*dy+y[0], color='blue', linewidth=0.8)
plt.tight_layout()
plt.show()

In [ ]:
from rtmag.process.download.dl_map import get_aia_map

In [ ]:
aia_map = get_aia_map(d, wavelength=171)

In [ ]:
aia_map.peek()

In [ ]:
import astropy.units as u

In [ ]:
aia_map_repro = aia_map.reproject_to(hmi_map.wcs)
aia_map_repro = aia_map_repro.resample([nx, ny] * u.pixel)

In [ ]:
aia_map_repro.peek()

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=2, figsize=(10, 6))
ax = axes[0]
ax.pcolormesh(x, y, aia_map_repro.data, 
              norm = aia_map_repro.plot_settings['norm'],
              cmap = aia_map_repro.plot_settings['cmap'],) 
ax.set_xlabel('x [Mm]')
ax.set_ylabel('y [Mm]')
ax.set_aspect('equal')

ax = axes[1]
ax.pcolormesh(x, y, bottom.T, cmap='gray', vmin=-2000, vmax=2000)
ax.set_xlabel('x [Mm]')
ax.set_ylabel('y [Mm]')
ax.set_aspect('equal')
for i, xl in enumerate(tracer_xs):
    assert seeds[i] in xl 
    ax.plot(xl[:,0]*dx+x[0], xl[:,1]*dy+y[0], color='blue', linewidth=0.8)

plt.tight_layout()
plt.show()

In [ ]:
import pyvista as pv

In [ ]:
xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')
mesh = pv.StructuredGrid(xv, yv, zv)
bx, by, bz = b[..., 0], b[..., 1], b[..., 2]
vectors = np.stack([bx, by, bz], axis=-1).transpose(2, 1, 0, 3).reshape(-1, 3)
mesh['vector'] = vectors
mesh.active_vectors_name = 'vector'
mesh

In [ ]:
xx, yy = np.meshgrid(x, y, indexing='ij')
seeds_xs = np.stack([xx[seeds[:, 0], seeds[:, 1]], 
                     yy[seeds[:, 0], seeds[:, 1]], 
                     np.zeros_like(seeds[:, 0])], axis=1)
seeds_xs.shape

In [ ]:
pv.global_theme.notebook = True
# pv.global_theme.jupyter_backend = 'static'
pv.global_theme.jupyter_backend = 'trame'

In [ ]:
p = pv.Plotter(window_size=(1000, 600))

p.show_bounds(
    grid='front',
    show_xlabels=False,
    show_ylabels=False,
    show_zlabels=False,
)
p.add_mesh(mesh.outline(), color='k')

seed = pv.PolyData(seeds_xs)
strl = mesh.streamlines_from_source(seed, vectors='vector', integration_direction='both',
                                    max_time=10000, initial_step_length=0.1)
p.add_mesh(strl.tube(radius=0.4), color='blue')

bottom_subset = mesh.extract_subset((0, nx-1, 0, ny-1, 0, 0)).extract_surface()
p.add_mesh(bottom_subset, cmap='gray', scalars='vector', component=2, clim=(-2000, 2000), 
           lighting=False, show_scalar_bar=False)

p.camera_position = "xy"
p.camera.roll = -30
p.camera.elevation = -70
p.show()